## <font color='green'> <div align="center">In the name of God </div></font>

### <font color='red'> Author: Sayed Kamaledin Ghiasi-Shrirazi</font> <a href="http://profsite.um.ac.ir/~k.ghiasi">(http://profsite.um.ac.ir/~k.ghiasi)</a> 

# A multi-layer feedforward neural network with Weight Sharing in PyTorch

### importing general modules

In [6]:
import numpy as np
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt

### importing PyTorch modules

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Defining the MLFF network by inheriting from nn.Module

Note that the following command is crucial:

``
self.layers = nn.ModuleList(layersList)
``

In [11]:
class MLFF(nn.Module):
    def __init__(self, num_hidden_layers, num_input_neurons, 
                 num_hidden_neurons, num_output_neurons):
        super().__init__()
        self.layersCount =  num_hidden_layers+1
        layersList = [None] * (self.layersCount)
        layersList[0] = nn.Linear(num_input_neurons, num_hidden_neurons)
        for i in range (1, num_hidden_layers):
            layersList[i] = nn.Linear(num_hidden_neurons, num_hidden_neurons)
        layersList[num_hidden_layers] = nn.Linear(num_hidden_neurons, num_output_neurons)
        self.layers  = nn.ModuleList(layersList)
        # Share all hidden layer weights
        for i in range (2, num_hidden_layers):
            layersList[i].weight = layersList[1].weight
            layersList[i].bias = layersList[1].bias
    
    def forward(self, x):
        for i in range(self.layersCount):
            x = self.layers[i](x)
            if (i < self.layersCount - 1):
                x = F.relu(x)
        return x

In [12]:
net = MLFF(5, 784, 100, 10)
print (net)

MLFF(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Linear(in_features=100, out_features=10, bias=True)
  )
)


### Choosing device

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cuda:0


## Loading MNIST

In [8]:
MnistTrainX = sio.loadmat ('../../datasets/mnist/MnistTrainX')['MnistTrainX'] / 255;
MnistTrainY = sio.loadmat ('../../datasets/mnist/MnistTrainY')['MnistTrainY'];
MnistTestX  = sio.loadmat ('../../datasets/mnist/MnistTestX')['MnistTestX'] / 255;
MnistTestY  = sio.loadmat ('../../datasets/mnist/MnistTestY')['MnistTestY'];

N = 60000
MnistTrainX = MnistTrainX[:N,:]
MnistTrainY = MnistTrainY[:N,:]
XTrain = MnistTrainX
yTrain = MnistTrainY.squeeze()
XTest = MnistTestX
yTest = MnistTestY.squeeze()
N, dim = XTrain.shape

## Optimization

In [14]:
num_epochs = 2
batch_size = 100
report_after_X_iterations = 100
learning_rate = 0.1
num_batches = N // batch_size

In [16]:
net.to(device)

optimizer = optim.SGD(net.parameters(), lr= learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range (num_epochs):
    for itr in range (num_batches):
        X = torch.tensor (MnistTrainX[itr*batch_size:(itr+1)*batch_size,:], dtype=torch.float)
        T = MnistTrainY[itr*batch_size:(itr+1)*batch_size]
        T = torch.tensor (T.squeeze(), dtype = torch.long)
        X = X.to(device)
        T = T.to(device)
        output = net(X)
        loss = criterion(output, T)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (itr % report_after_X_iterations == 0):
            print('\n---- iteration #{0} of {1} at epoch #{2} of {3} ---- :'.format(
                itr, num_batches, epoch, num_epochs))
            score = 0.0
            for i in range(num_batches):
                X = MnistTrainX[i * batch_size:(i + 1) * batch_size, :]
                T = MnistTrainY[i * batch_size:(i + 1) * batch_size]
                T = T.squeeze()
                X = torch.tensor(X, dtype=torch.float).to(device)
                #T = torch.tensor(T, dtype=torch.long).to(device)
                output = net(X)
                prediction = torch.argmax(output, dim=1).cpu().numpy()
                score += np.sum(prediction == T)
            score /= N
            score *= 100
            print('Loss = {0}, Accuracy on training data = {1}%'.format(loss.item(), score))


---- iteration #0 of 600 at epoch #0 of 2 ---- :
Loss = 2.303187847137451, Accuracy on training data = 10.218333333333334%

---- iteration #100 of 600 at epoch #0 of 2 ---- :
Loss = 2.299666404724121, Accuracy on training data = 11.236666666666666%

---- iteration #200 of 600 at epoch #0 of 2 ---- :
Loss = 2.294888973236084, Accuracy on training data = 11.236666666666666%

---- iteration #300 of 600 at epoch #0 of 2 ---- :
Loss = 2.2778592109680176, Accuracy on training data = 16.181666666666665%

---- iteration #400 of 600 at epoch #0 of 2 ---- :
Loss = 1.1601697206497192, Accuracy on training data = 54.56166666666667%

---- iteration #500 of 600 at epoch #0 of 2 ---- :
Loss = 0.6362378001213074, Accuracy on training data = 78.55166666666666%

---- iteration #0 of 600 at epoch #1 of 2 ---- :
Loss = 0.4121897518634796, Accuracy on training data = 86.30833333333334%

---- iteration #100 of 600 at epoch #1 of 2 ---- :
Loss = 0.3385835587978363, Accuracy on training data = 91.42666666666